# load pkgs

In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
import PipelineTS
PipelineTS.__version__

'0.3.8'

In [3]:
from PipelineTS.dataset import LoadMessagesSentDataSets
from PipelineTS.io import save_model, load_model

In [4]:
import pandas as pd
time_col = 'date'
target_col = 'ta'

init_data = LoadMessagesSentDataSets()[[time_col, target_col]]
init_data[time_col] = pd.to_datetime(init_data[time_col])

In [5]:
accelerator = 'cpu'
lags = 60
n = 40
quantile = 0.9

In [6]:
test_data = init_data.iloc[-n:, :]

train_data = init_data.iloc[:-n, :]

train_data.tail(5)

,date,ta
557,2021-07-11,125957
558,2021-07-12,139012
559,2021-07-13,127014
560,2021-07-14,118241
561,2021-07-15,114358


# Preprocessing

In [7]:
from PipelineTS.metrics import quantile_acc
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from spinesTS.metrics import wmape

from PipelineTS.preprocessing import Scaler

In [8]:
# use a scaler to scale data
valid_data = test_data.copy()
data = train_data.copy()

scaler = Scaler(scaler_name='min_max')

data[target_col] = scaler.fit_transform(train_data[target_col].values.reshape(-1, 1)).squeeze()
valid_data[target_col] = scaler.transform(test_data[target_col].values.reshape(-1, 1)).squeeze()

In [9]:
def inverse_transform_prediction(df, scaler):
    """define a function to inverse data."""
    for i in df.columns:
        if i != time_col:
            df[i] = scaler.inverse_transform(df[i].values.reshape(-1, 1)).squeeze()

In [10]:
def prediction_metrics(true_df, pred_df):
    """define a function to evaluate the performance of model."""
    yt = true_df[target_col].values
    res = {}
    for i in pred_df.columns:
        if i == target_col:
            res['mae'] = mean_absolute_error(yt.reshape(-1, 1), pred_df[i].values.reshape(-1, 1))
            res['mape'] = mean_absolute_percentage_error(yt.reshape(-1, 1), pred_df[i].values.reshape(-1, 1))
            res['wmape'] = wmape(yt.reshape(-1, 1), pred_df[i].values.reshape(-1, 1))
            
    if target_col+'_lower' in pred_df.columns and target_col+'_upper' in pred_df.columns:
        res['quantile_acc'] = quantile_acc(yt, pred_df[target_col+'_lower'].values, pred_df[target_col+'_upper'].values)

    return res

In [11]:
def test_load_and_save_model(fp, model, scaler):
    save_model(fp, model, scaler=scaler)
    model, scaler = load_model(fp)
    res = model.predict(n)
    inverse_transform_prediction(res, scaler)
    print(prediction_metrics(test_data, res))
    return res

# WideGBRTModel

In [12]:
from PipelineTS.ml_model import WideGBRTModel

model = WideGBRTModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                          quantile=quantile)
model.fit(data)

In [13]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 33138.30730011646, 'mape': 0.29808815747191425, 'wmape': 0.2626567117814881, 'quantile_acc': 0.4}


,date,ta,ta_lower,ta_upper
0,2021-07-16,133950.588794,111500.372650,156400.804937
1,2021-07-17,148384.796920,122571.505563,174198.088276
2,2021-07-18,151408.932290,124891.037355,177926.827225
3,2021-07-19,138900.553243,115297.028058,162504.078428
4,2021-07-20,142183.137183,117814.791608,166551.482757
5,2021-07-21,146655.685171,121245.265439,172066.104904
6,2021-07-22,144268.542823,119414.311500,169122.774146
7,2021-07-23,137330.083550,114092.467436,160567.699663
8,2021-07-24,140248.872227,116331.197619,164166.546836
9,2021-07-25,127643.172049,106662.542372,148623.801727


In [14]:
test_load_and_save_model('wide_gbrt.zip', model, scaler)

{'mae': 33138.30730011646, 'mape': 0.29808815747191425, 'wmape': 0.2626567117814881, 'quantile_acc': 0.4}


,date,ta,ta_lower,ta_upper
0,2021-07-16,133950.588794,111500.372650,156400.804937
1,2021-07-17,148384.796920,122571.505563,174198.088276
2,2021-07-18,151408.932290,124891.037355,177926.827225
3,2021-07-19,138900.553243,115297.028058,162504.078428
4,2021-07-20,142183.137183,117814.791608,166551.482757
5,2021-07-21,146655.685171,121245.265439,172066.104904
6,2021-07-22,144268.542823,119414.311500,169122.774146
7,2021-07-23,137330.083550,114092.467436,160567.699663
8,2021-07-24,140248.872227,116331.197619,164166.546836
9,2021-07-25,127643.172049,106662.542372,148623.801727


# ProphetModel

In [15]:
from PipelineTS.statistic_model import ProphetModel
model = ProphetModel(time_col=time_col, target_col=target_col, lags=lags, quantile=quantile)

In [16]:
model.fit(data)

In [17]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 20629.31101318634, 'mape': 0.15710029391827462, 'wmape': 0.16350946799935434, 'quantile_acc': 0.825}


,date,ta,ta_lower,ta_upper
0,2021-07-16,120837.635750,88919.855405,152755.416096
1,2021-07-17,124247.455529,91022.245671,157472.665388
2,2021-07-18,116144.893994,86026.454806,146263.333181
3,2021-07-19,129787.642383,94438.154918,165137.129849
4,2021-07-20,133567.600165,96768.760921,170366.439410
5,2021-07-21,127238.696055,92866.552811,161610.839300
6,2021-07-22,126495.277287,92408.183606,160582.370969
7,2021-07-23,118458.236451,87452.790795,149463.682108
8,2021-07-24,121868.056230,89555.181061,154180.931400
9,2021-07-25,113765.494695,84559.390196,142971.599194


In [18]:
test_load_and_save_model('prophet.zip', model, scaler)

{'mae': 20629.31101318634, 'mape': 0.15710029391827462, 'wmape': 0.16350946799935434, 'quantile_acc': 0.825}


,date,ta,ta_lower,ta_upper
0,2021-07-16,120837.635750,88919.855405,152755.416096
1,2021-07-17,124247.455529,91022.245671,157472.665388
2,2021-07-18,116144.893994,86026.454806,146263.333181
3,2021-07-19,129787.642383,94438.154918,165137.129849
4,2021-07-20,133567.600165,96768.760921,170366.439410
5,2021-07-21,127238.696055,92866.552811,161610.839300
6,2021-07-22,126495.277287,92408.183606,160582.370969
7,2021-07-23,118458.236451,87452.790795,149463.682108
8,2021-07-24,121868.056230,89555.181061,154180.931400
9,2021-07-25,113765.494695,84559.390196,142971.599194


# AutoARIMA

In [19]:
from PipelineTS.statistic_model import AutoARIMAModel

In [20]:
model = AutoARIMAModel(time_col=time_col, target_col=target_col, lags=lags, quantile=quantile)
model.fit(data=data)

In [21]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 26842.097945433256, 'mape': 0.23706463101016176, 'wmape': 0.2127524836985068, 'quantile_acc': 0.35}


,date,ta,ta_lower,ta_upper
0,2021-07-16,129765.757857,111582.766386,147948.749329
1,2021-07-17,131241.459259,112767.348710,149715.569808
2,2021-07-18,135444.346075,116141.110791,154747.581358
3,2021-07-19,136134.389346,116695.025737,155573.752955
4,2021-07-20,139506.742460,119402.097699,159611.387220
5,2021-07-21,139653.891874,119520.218205,159787.565544
6,2021-07-22,142410.544260,121733.051835,163088.036684
7,2021-07-23,142187.765018,121554.221381,162821.308655
8,2021-07-24,144483.920625,123397.402761,165570.438490
9,2021-07-25,144014.180480,123020.330629,165008.030331


In [22]:
test_load_and_save_model('auto_arima.zip', model, scaler)

{'mae': 26842.097945433256, 'mape': 0.23706463101016176, 'wmape': 0.2127524836985068, 'quantile_acc': 0.35}


,date,ta,ta_lower,ta_upper
0,2021-07-16,129765.757857,111582.766386,147948.749329
1,2021-07-17,131241.459259,112767.348710,149715.569808
2,2021-07-18,135444.346075,116141.110791,154747.581358
3,2021-07-19,136134.389346,116695.025737,155573.752955
4,2021-07-20,139506.742460,119402.097699,159611.387220
5,2021-07-21,139653.891874,119520.218205,159787.565544
6,2021-07-22,142410.544260,121733.051835,163088.036684
7,2021-07-23,142187.765018,121554.221381,162821.308655
8,2021-07-24,144483.920625,123397.402761,165570.438490
9,2021-07-25,144014.180480,123020.330629,165008.030331


# GBDT

In [23]:
from PipelineTS.ml_model import *

## CatBoost

In [24]:
model = CatBoostModel(time_col=time_col, target_col=target_col, lags=30, random_state=42, quantile=quantile)
model.fit(data=data)

In [25]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 31336.234903759963, 'mape': 0.28290025124066653, 'wmape': 0.24837335066311497, 'quantile_acc': 0.375}


,date,ta,ta_lower,ta_upper
0,2021-07-16,142594.803098,118294.980598,166894.625598
1,2021-07-17,140851.167848,116954.870172,164747.465523
2,2021-07-18,144692.676044,119907.348442,169478.003647
3,2021-07-19,153041.676906,126324.162451,179759.191360
4,2021-07-20,147159.892963,121803.583973,172516.201953
5,2021-07-21,142521.701536,118238.796735,166804.606338
6,2021-07-22,159627.049734,131385.500128,187868.599340
7,2021-07-23,155479.357046,128197.696822,182761.017270
8,2021-07-24,151356.867329,125029.263831,177684.470826
9,2021-07-25,146740.268624,121481.072169,171999.465080


In [26]:
test_load_and_save_model('catboost.zip', model, scaler)

{'mae': 31336.234903759963, 'mape': 0.28290025124066653, 'wmape': 0.24837335066311497, 'quantile_acc': 0.375}


,date,ta,ta_lower,ta_upper
0,2021-07-16,142594.803098,118294.980598,166894.625598
1,2021-07-17,140851.167848,116954.870172,164747.465523
2,2021-07-18,144692.676044,119907.348442,169478.003647
3,2021-07-19,153041.676906,126324.162451,179759.191360
4,2021-07-20,147159.892963,121803.583973,172516.201953
5,2021-07-21,142521.701536,118238.796735,166804.606338
6,2021-07-22,159627.049734,131385.500128,187868.599340
7,2021-07-23,155479.357046,128197.696822,182761.017270
8,2021-07-24,151356.867329,125029.263831,177684.470826
9,2021-07-25,146740.268624,121481.072169,171999.465080


## XGBoost

In [27]:
model = XGBoostModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)
model.fit(data=data)

In [28]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 34624.0408203125, 'mape': 0.30387842439836577, 'wmape': 0.274432747215768, 'quantile_acc': 0.375}


,date,ta,ta_lower,ta_upper
0,2021-07-16,124142.789062,103798.226562,144487.375000
1,2021-07-17,144761.984375,119570.507812,169953.468750
2,2021-07-18,149495.890625,123191.632812,175800.156250
3,2021-07-19,147176.281250,121417.289062,172935.281250
4,2021-07-20,130929.562500,108989.640625,152869.468750
5,2021-07-21,148334.734375,122303.414062,174366.046875
6,2021-07-22,143281.593750,118438.117188,168125.078125
7,2021-07-23,162336.359375,133013.718750,191659.000000
8,2021-07-24,153850.000000,126522.234375,181177.781250
9,2021-07-25,176169.390625,143595.046875,208743.734375


In [29]:
test_load_and_save_model('xgboost.zip', model, scaler)

{'mae': 34624.0408203125, 'mape': 0.30387842439836577, 'wmape': 0.274432747215768, 'quantile_acc': 0.375}


,date,ta,ta_lower,ta_upper
0,2021-07-16,124142.789062,103798.226562,144487.375000
1,2021-07-17,144761.984375,119570.507812,169953.468750
2,2021-07-18,149495.890625,123191.632812,175800.156250
3,2021-07-19,147176.281250,121417.289062,172935.281250
4,2021-07-20,130929.562500,108989.640625,152869.468750
5,2021-07-21,148334.734375,122303.414062,174366.046875
6,2021-07-22,143281.593750,118438.117188,168125.078125
7,2021-07-23,162336.359375,133013.718750,191659.000000
8,2021-07-24,153850.000000,126522.234375,181177.781250
9,2021-07-25,176169.390625,143595.046875,208743.734375


## LightGBM

In [30]:
model = LightGBMModel(time_col=time_col, target_col=target_col, lags=10, random_state=42, quantile=quantile, verbose=-1)
model.fit(data=data)

In [31]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 34893.50895835457, 'mape': 0.3018012609808552, 'wmape': 0.2765685719103432, 'quantile_acc': 0.5}


,date,ta,ta_lower,ta_upper
0,2021-07-16,126356.556376,103346.041074,149367.071679
1,2021-07-17,125503.132872,102713.858785,148292.406960
2,2021-07-18,105144.323616,87632.860228,122655.787004
3,2021-07-19,102551.364857,85712.099205,119390.630510
4,2021-07-20,104909.650870,87459.023964,122360.277777
5,2021-07-21,153957.802806,123791.949886,184123.655726
6,2021-07-22,227806.900082,178496.432620,277117.367544
7,2021-07-23,133300.269892,108489.668689,158110.871094
8,2021-07-24,207146.728008,163192.196439,251101.259578
9,2021-07-25,161333.074333,129255.258563,193410.890104


In [32]:
test_load_and_save_model('lightgbm.zip', model, scaler)

{'mae': 34893.50895835457, 'mape': 0.3018012609808552, 'wmape': 0.2765685719103432, 'quantile_acc': 0.5}


,date,ta,ta_lower,ta_upper
0,2021-07-16,126356.556376,103346.041074,149367.071679
1,2021-07-17,125503.132872,102713.858785,148292.406960
2,2021-07-18,105144.323616,87632.860228,122655.787004
3,2021-07-19,102551.364857,85712.099205,119390.630510
4,2021-07-20,104909.650870,87459.023964,122360.277777
5,2021-07-21,153957.802806,123791.949886,184123.655726
6,2021-07-22,227806.900082,178496.432620,277117.367544
7,2021-07-23,133300.269892,108489.668689,158110.871094
8,2021-07-24,207146.728008,163192.196439,251101.259578
9,2021-07-25,161333.074333,129255.258563,193410.890104


## RandomForest

In [33]:
model = RandomForestModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)
model.fit(data=data)

In [34]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 33039.98048888488, 'mape': 0.29682306190333463, 'wmape': 0.26187736609300283, 'quantile_acc': 0.35}


,date,ta,ta_lower,ta_upper
0,2021-07-16,144078.240705,121725.219363,166431.262048
1,2021-07-17,142339.410336,120351.405236,164327.415437
2,2021-07-18,147687.040013,124576.457505,170797.622521
3,2021-07-19,150610.920475,126886.555348,174335.285602
4,2021-07-20,150582.860260,126864.385549,174301.334971
5,2021-07-21,154313.539773,129811.918735,178815.160810
6,2021-07-22,152286.679482,128210.538089,176362.820876
7,2021-07-23,151232.699507,127377.810178,175087.588836
8,2021-07-24,151913.629707,127915.799131,175911.460284
9,2021-07-25,154126.679958,129664.284637,178589.075279


In [35]:
test_load_and_save_model('rf.zip', model, scaler)

{'mae': 33039.98048888488, 'mape': 0.29682306190333463, 'wmape': 0.26187736609300283, 'quantile_acc': 0.35}


,date,ta,ta_lower,ta_upper
0,2021-07-16,144078.240705,121725.219363,166431.262048
1,2021-07-17,142339.410336,120351.405236,164327.415437
2,2021-07-18,147687.040013,124576.457505,170797.622521
3,2021-07-19,150610.920475,126886.555348,174335.285602
4,2021-07-20,150582.860260,126864.385549,174301.334971
5,2021-07-21,154313.539773,129811.918735,178815.160810
6,2021-07-22,152286.679482,128210.538089,176362.820876
7,2021-07-23,151232.699507,127377.810178,175087.588836
8,2021-07-24,151913.629707,127915.799131,175911.460284
9,2021-07-25,154126.679958,129664.284637,178589.075279


# NN Model

In [36]:
from PipelineTS.nn_model import *

## DLinearModel

In [37]:
model = DLinearModel(time_col=time_col, target_col=target_col, lags=10, random_state=42, quantile=quantile)

In [38]:
model.fit(data)

In [39]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 37295.61171875, 'mape': 0.3305806304685669, 'wmape': 0.2956078187461187, 'quantile_acc': 0.325}


,date,ta,ta_lower,ta_upper
0,2021-07-16,128949.164062,110120.492188,147777.843750
1,2021-07-17,121173.328125,103947.312500,138399.343750
2,2021-07-18,119036.164062,102250.632812,135821.703125
3,2021-07-19,149312.281250,126286.632812,172337.906250
4,2021-07-20,152158.593750,128546.312500,175770.875000
5,2021-07-21,154106.203125,130092.507812,178119.906250
6,2021-07-22,144746.671875,122662.031250,166831.296875
7,2021-07-23,151426.796875,127965.335938,174888.250000
8,2021-07-24,151416.859375,127957.460938,174876.281250
9,2021-07-25,165733.203125,139323.109375,192143.296875


In [40]:
test_load_and_save_model('dlinear.zip', model, scaler)

{'mae': 37295.61171875, 'mape': 0.3305806304685669, 'wmape': 0.2956078187461187, 'quantile_acc': 0.325}


,date,ta,ta_lower,ta_upper
0,2021-07-16,128949.164062,110120.492188,147777.843750
1,2021-07-17,121173.328125,103947.312500,138399.343750
2,2021-07-18,119036.164062,102250.632812,135821.703125
3,2021-07-19,149312.281250,126286.632812,172337.906250
4,2021-07-20,152158.593750,128546.312500,175770.875000
5,2021-07-21,154106.203125,130092.507812,178119.906250
6,2021-07-22,144746.671875,122662.031250,166831.296875
7,2021-07-23,151426.796875,127965.335938,174888.250000
8,2021-07-24,151416.859375,127957.460938,174876.281250
9,2021-07-25,165733.203125,139323.109375,192143.296875


##  NLinearModel

In [41]:
model = NLinearModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)

In [42]:
model.fit(data)
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 37330.14375, 'mape': 0.33019139644534007, 'wmape': 0.29588152221857184, 'quantile_acc': 0.35}


,date,ta,ta_lower,ta_upper
0,2021-07-16,146164.062500,117308.320312,175019.812500
1,2021-07-17,144569.000000,116137.187500,173000.828125
2,2021-07-18,151201.578125,121006.937500,181396.203125
3,2021-07-19,158202.781250,126147.343750,190258.218750
4,2021-07-20,173455.906250,137346.468750,209565.359375
5,2021-07-21,166265.281250,132066.984375,200463.593750
6,2021-07-22,165571.312500,131557.453125,199585.171875
7,2021-07-23,167622.000000,133063.109375,202180.890625
8,2021-07-24,164007.281250,130409.117188,197605.437500
9,2021-07-25,165456.531250,131473.171875,199439.890625


In [43]:
test_load_and_save_model('nlinear.zip', model, scaler)

{'mae': 37330.14375, 'mape': 0.33019139644534007, 'wmape': 0.29588152221857184, 'quantile_acc': 0.35}


,date,ta,ta_lower,ta_upper
0,2021-07-16,146164.062500,117308.320312,175019.812500
1,2021-07-17,144569.000000,116137.187500,173000.828125
2,2021-07-18,151201.578125,121006.937500,181396.203125
3,2021-07-19,158202.781250,126147.343750,190258.218750
4,2021-07-20,173455.906250,137346.468750,209565.359375
5,2021-07-21,166265.281250,132066.984375,200463.593750
6,2021-07-22,165571.312500,131557.453125,199585.171875
7,2021-07-23,167622.000000,133063.109375,202180.890625
8,2021-07-24,164007.281250,130409.117188,197605.437500
9,2021-07-25,165456.531250,131473.171875,199439.890625


##  NBeatsModel 

In [44]:
model = NBeatsModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)

In [45]:
model.fit(data)
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 19584.6986328125, 'mape': 0.16375120592530434, 'wmape': 0.155229791839967, 'quantile_acc': 0.675}


,date,ta,ta_lower,ta_upper
0,2021-07-16,135736.046875,111827.015625,159645.078125
1,2021-07-17,140940.515625,115763.585938,166117.468750
2,2021-07-18,144399.031250,118379.523438,170418.515625
3,2021-07-19,151064.578125,123421.218750,178707.937500
4,2021-07-20,152192.406250,124274.289062,180110.531250
5,2021-07-21,140528.234375,115451.742188,165604.734375
6,2021-07-22,117658.117188,98153.218750,137163.015625
7,2021-07-23,126494.007812,104836.515625,148151.500000
8,2021-07-24,113770.664062,95212.812500,132328.500000
9,2021-07-25,118338.140625,98667.578125,138008.718750


In [46]:
test_load_and_save_model('nbeats.zip', model, scaler)

{'mae': 19584.6986328125, 'mape': 0.16375120592530434, 'wmape': 0.155229791839967, 'quantile_acc': 0.675}


,date,ta,ta_lower,ta_upper
0,2021-07-16,135736.046875,111827.015625,159645.078125
1,2021-07-17,140940.515625,115763.585938,166117.468750
2,2021-07-18,144399.031250,118379.523438,170418.515625
3,2021-07-19,151064.578125,123421.218750,178707.937500
4,2021-07-20,152192.406250,124274.289062,180110.531250
5,2021-07-21,140528.234375,115451.742188,165604.734375
6,2021-07-22,117658.117188,98153.218750,137163.015625
7,2021-07-23,126494.007812,104836.515625,148151.500000
8,2021-07-24,113770.664062,95212.812500,132328.500000
9,2021-07-25,118338.140625,98667.578125,138008.718750


## TCNModel

In [47]:
model = TCNModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)

In [48]:
model.fit(data)
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 49130.300390625, 'mape': 0.4364507445715297, 'wmape': 0.3894104497423431, 'quantile_acc': 0.25}


,date,ta,ta_lower,ta_upper
0,2021-07-16,182758.828125,143238.312500,222279.343750
1,2021-07-17,187316.453125,146555.140625,228077.781250
2,2021-07-18,183686.515625,143913.437500,223459.593750
3,2021-07-19,182573.234375,143103.234375,222043.203125
4,2021-07-20,182450.515625,143013.937500,221887.109375
5,2021-07-21,182437.609375,143004.546875,221870.671875
6,2021-07-22,182518.625000,143063.500000,221973.734375
7,2021-07-23,176197.140625,138463.031250,213931.250000
8,2021-07-24,177706.546875,139561.500000,215851.578125
9,2021-07-25,175353.500000,137849.062500,212857.937500


In [49]:
test_load_and_save_model('tcn.zip', model, scaler)

{'mae': 49130.300390625, 'mape': 0.4364507445715297, 'wmape': 0.3894104497423431, 'quantile_acc': 0.25}


,date,ta,ta_lower,ta_upper
0,2021-07-16,182758.828125,143238.312500,222279.343750
1,2021-07-17,187316.453125,146555.140625,228077.781250
2,2021-07-18,183686.515625,143913.437500,223459.593750
3,2021-07-19,182573.234375,143103.234375,222043.203125
4,2021-07-20,182450.515625,143013.937500,221887.109375
5,2021-07-21,182437.609375,143004.546875,221870.671875
6,2021-07-22,182518.625000,143063.500000,221973.734375
7,2021-07-23,176197.140625,138463.031250,213931.250000
8,2021-07-24,177706.546875,139561.500000,215851.578125
9,2021-07-25,175353.500000,137849.062500,212857.937500


## NHitsModel

In [50]:
model = NHitsModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)

In [51]:
model.fit(data)
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 25226.28671875, 'mape': 0.22483657232602852, 'wmape': 0.19994544259599567, 'quantile_acc': 0.525}


,date,ta,ta_lower,ta_upper
0,2021-07-16,129828.273438,108896.765625,150759.796875
1,2021-07-17,132439.375000,110915.289062,153963.453125
2,2021-07-18,142921.609375,119018.671875,166824.546875
3,2021-07-19,144612.968750,120326.203125,168899.750000
4,2021-07-20,158269.671875,130883.632812,185655.718750
5,2021-07-21,151263.343750,125467.335938,177059.359375
6,2021-07-22,143454.890625,119430.937500,167478.843750
7,2021-07-23,137749.640625,115020.445312,160478.843750
8,2021-07-24,123837.632812,104265.640625,143409.625000
9,2021-07-25,126111.648438,106023.585938,146199.718750


In [52]:
test_load_and_save_model('nhits.zip', model, scaler)

{'mae': 25226.28671875, 'mape': 0.22483657232602852, 'wmape': 0.19994544259599567, 'quantile_acc': 0.525}


,date,ta,ta_lower,ta_upper
0,2021-07-16,129828.273438,108896.765625,150759.796875
1,2021-07-17,132439.375000,110915.289062,153963.453125
2,2021-07-18,142921.609375,119018.671875,166824.546875
3,2021-07-19,144612.968750,120326.203125,168899.750000
4,2021-07-20,158269.671875,130883.632812,185655.718750
5,2021-07-21,151263.343750,125467.335938,177059.359375
6,2021-07-22,143454.890625,119430.937500,167478.843750
7,2021-07-23,137749.640625,115020.445312,160478.843750
8,2021-07-24,123837.632812,104265.640625,143409.625000
9,2021-07-25,126111.648438,106023.585938,146199.718750


## TFTModel

In [53]:
model = TFTModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)

In [54]:
model.fit(data)
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 34548.78046875, 'mape': 0.30836444085253706, 'wmape': 0.27383622801851687, 'quantile_acc': 0.35}


,date,ta,ta_lower,ta_upper
0,2021-07-16,133110.140625,109424.492188,156795.796875
1,2021-07-17,137971.187500,113080.109375,162862.265625
2,2021-07-18,141676.312500,115866.437500,167486.187500
3,2021-07-19,143906.781250,117543.796875,170269.750000
4,2021-07-20,145410.656250,118674.750000,172146.578125
5,2021-07-21,146605.781250,119573.507812,173638.062500
6,2021-07-22,147680.187500,120381.492188,174978.906250
7,2021-07-23,148715.156250,121159.804688,176270.515625
8,2021-07-24,149742.812500,121932.617188,177553.000000
9,2021-07-25,150773.062500,122707.390625,178838.718750


In [55]:
test_load_and_save_model('tft.zip', model, scaler)

{'mae': 34548.78046875, 'mape': 0.30836444085253706, 'wmape': 0.27383622801851687, 'quantile_acc': 0.35}


,date,ta,ta_lower,ta_upper
0,2021-07-16,133110.140625,109424.492188,156795.796875
1,2021-07-17,137971.187500,113080.109375,162862.265625
2,2021-07-18,141676.312500,115866.437500,167486.187500
3,2021-07-19,143906.781250,117543.796875,170269.750000
4,2021-07-20,145410.656250,118674.750000,172146.578125
5,2021-07-21,146605.781250,119573.507812,173638.062500
6,2021-07-22,147680.187500,120381.492188,174978.906250
7,2021-07-23,148715.156250,121159.804688,176270.515625
8,2021-07-24,149742.812500,121932.617188,177553.000000
9,2021-07-25,150773.062500,122707.390625,178838.718750


## GAU

In [56]:
from PipelineTS.nn_model import *

In [57]:
model = GAUModel(time_col=time_col, target_col='ta', lags=lags, random_state=42, 
                     quantile=quantile, accelerator=accelerator, patience=10, verbose=True)
model.fit(data)

MPS  available: True   | MPS  use: False
CUDA available: False  | CUDA use: False
TPU  available: False  | TPU  use: False
CPU  available: True   | CPU  use: True


Epoch 1/1000  
1/1 - p0 -  [*lr: 0.0009045] - loss: 0.6981 - mae: 0.6981 - val_loss: 0.6322 - val_mae: 0.6322 - 0.02s/epoch - 0.020s/step
Epoch 2/1000  
1/1 - p0 -  [*lr: 0.0006545] - loss: 0.6322 - mae: 0.6322 - val_loss: 0.5778 - val_mae: 0.5778 - 0.01s/epoch - 0.015s/step
Epoch 3/1000  
1/1 - p0 -  [*lr: 0.0003455] - loss: 0.5778 - mae: 0.5778 - val_loss: 0.5411 - val_mae: 0.5411 - 0.02s/epoch - 0.015s/step
Epoch 4/1000  
1/1 - p0 -  [*lr: 9.55e-05] - loss: 0.5411 - mae: 0.5411 - val_loss: 0.5221 - val_mae: 0.5221 - 0.01s/epoch - 0.015s/step
Epoch 5/1000  
1/1 - p0 - [*lr: 0.000000e+00] - loss: 0.5221 - mae: 0.5221 - val_loss: 0.5168 - val_mae: 0.5168 - 0.01s/epoch - 0.014s/step
Epoch 6/1000  
1/1 - p1 -  [*lr: 9.55e-05] - loss: 0.5168 - mae: 0.5168 - val_loss: 0.5168 - val_mae: 0.5168 - 0.01s/epoch - 0.014s/step
Epoch 7/1000  
1/1 - p0 -  [*lr: 0.0003455] - loss: 0.5168 - mae: 0.5168 - val_loss: 0.5115 - val_mae: 0.5115 - 0.01s/epoch - 0.015s/step
Epoch 8/1000  
1/1 - p0 -  [*lr: 0

In [58]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(valid_data, res))
res

{'mae': 172319.20182608554, 'mape': 1924610.7288178224, 'wmape': 1780328.0190966565, 'quantile_acc': 0.0}


,date,ta,ta_lower,ta_upper
0,2021-07-16,170221.632730,132215.102403,208228.163057
1,2021-07-17,184860.001270,142658.584996,227061.417544
2,2021-07-18,178946.486706,138439.693646,219453.279767
3,2021-07-19,178080.197369,137821.654990,218338.739747
4,2021-07-20,163965.684959,127751.908197,200179.461720
5,2021-07-21,161047.673504,125670.105041,196425.241968
6,2021-07-22,214693.892846,163943.042106,265444.743586
7,2021-07-23,179209.641984,138627.437079,219791.846889
8,2021-07-24,140956.704163,111336.561790,170576.846536
9,2021-07-25,131161.979923,104348.690715,157975.269131


In [59]:
test_load_and_save_model('gau.zip', model, scaler)

{'mae': 50302.7193048954, 'mape': 0.44377035612343363, 'wmape': 0.3987031300854819, 'quantile_acc': 0.3}


,date,ta,ta_lower,ta_upper
0,2021-07-16,170221.632730,132215.102403,208228.163057
1,2021-07-17,184860.001270,142658.584996,227061.417544
2,2021-07-18,178946.486706,138439.693646,219453.279767
3,2021-07-19,178080.197369,137821.654990,218338.739747
4,2021-07-20,163965.684959,127751.908197,200179.461720
5,2021-07-21,161047.673504,125670.105041,196425.241968
6,2021-07-22,214693.892846,163943.042106,265444.743586
7,2021-07-23,179209.641984,138627.437079,219791.846889
8,2021-07-24,140956.704163,111336.561790,170576.846536
9,2021-07-25,131161.979923,104348.690715,157975.269131


## StackingRNN

In [60]:
model = StackingRNNModel(time_col=time_col, target_col=target_col, lags=10, random_state=42, 
                     quantile=quantile, accelerator=accelerator, patience=100, verbose=True)
model.fit(data)

MPS  available: True   | MPS  use: False
CUDA available: False  | CUDA use: False
TPU  available: False  | TPU  use: False
CPU  available: True   | CPU  use: True


Epoch 1/1000  
1/1 - p0 -  [*lr: 0.0009045] - loss: 0.1735 - mae: 0.1735 - val_loss: 0.1254 - val_mae: 0.1254 - 0.01s/epoch - 0.009s/step
Epoch 2/1000  
1/1 - p0 -  [*lr: 0.0006545] - loss: 0.1254 - mae: 0.1254 - val_loss: 0.0876 - val_mae: 0.0876 - 0.01s/epoch - 0.008s/step
Epoch 3/1000  
1/1 - p0 -  [*lr: 0.0003455] - loss: 0.0876 - mae: 0.0876 - val_loss: 0.0672 - val_mae: 0.0672 - 0.01s/epoch - 0.010s/step
Epoch 4/1000  
1/1 - p0 -  [*lr: 9.55e-05] - loss: 0.0672 - mae: 0.0672 - val_loss: 0.0595 - val_mae: 0.0595 - 0.01s/epoch - 0.009s/step
Epoch 5/1000  
1/1 - p0 - [*lr: 0.000000e+00] - loss: 0.0595 - mae: 0.0595 - val_loss: 0.0577 - val_mae: 0.0577 - 0.01s/epoch - 0.009s/step
Epoch 6/1000  
1/1 - p1 -  [*lr: 9.55e-05] - loss: 0.0577 - mae: 0.0577 - val_loss: 0.0577 - val_mae: 0.0577 - 0.01s/epoch - 0.008s/step
Epoch 7/1000  
1/1 - p0 -  [*lr: 0.0003455] - loss: 0.0577 - mae: 0.0577 - val_loss: 0.0561 - val_mae: 0.0561 - 0.01s/epoch - 0.009s/step
Epoch 8/1000  
1/1 - p0 -  [*lr: 0

In [61]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 32857.45145122185, 'mape': 0.29346660543856506, 'wmape': 0.2604306272356732, 'quantile_acc': 0.25}


,date,ta,ta_lower,ta_upper
0,2021-07-16,135926.482649,118939.272974,152913.692324
1,2021-07-17,136278.651743,119230.603259,153326.700227
2,2021-07-18,137298.676724,120074.414298,154522.939150
3,2021-07-19,140116.718081,122405.626227,157827.809934
4,2021-07-20,137498.439656,120239.667277,154757.212034
5,2021-07-21,141743.347406,123751.247958,159735.446855
6,2021-07-22,149965.071509,130552.631927,169377.511091
7,2021-07-23,145382.973475,126762.112113,164003.834836
8,2021-07-24,152230.560416,132426.747349,172034.373483
9,2021-07-25,147442.392941,128465.757525,166419.028358


In [62]:
test_load_and_save_model('stacking_rnn.zip', model, scaler)

{'mae': 32857.45145122185, 'mape': 0.29346660543856506, 'wmape': 0.2604306272356732, 'quantile_acc': 0.25}


,date,ta,ta_lower,ta_upper
0,2021-07-16,135926.482649,118939.272974,152913.692324
1,2021-07-17,136278.651743,119230.603259,153326.700227
2,2021-07-18,137298.676724,120074.414298,154522.939150
3,2021-07-19,140116.718081,122405.626227,157827.809934
4,2021-07-20,137498.439656,120239.667277,154757.212034
5,2021-07-21,141743.347406,123751.247958,159735.446855
6,2021-07-22,149965.071509,130552.631927,169377.511091
7,2021-07-23,145382.973475,126762.112113,164003.834836
8,2021-07-24,152230.560416,132426.747349,172034.373483
9,2021-07-25,147442.392941,128465.757525,166419.028358


## Time2Vec

In [63]:
model = Time2VecModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                     quantile=quantile, accelerator=accelerator, patience=10, verbose=True)
model.fit(data)

MPS  available: True   | MPS  use: False
CUDA available: False  | CUDA use: False
TPU  available: False  | TPU  use: False
CPU  available: True   | CPU  use: True


Epoch 1/1000  
1/1 - p0 -  [*lr: 0.0090451] - loss: 0.4295 - mae: 0.4295 - val_loss: 0.4087 - val_mae: 0.4087 - 0.04s/epoch - 0.040s/step
Epoch 2/1000  
1/1 - p0 -  [*lr: 0.0065451] - loss: 0.4087 - mae: 0.4087 - val_loss: 0.3538 - val_mae: 0.3538 - 0.04s/epoch - 0.042s/step
Epoch 3/1000  
1/1 - p0 -  [*lr: 0.0034549] - loss: 0.3538 - mae: 0.3538 - val_loss: 0.2289 - val_mae: 0.2289 - 0.04s/epoch - 0.040s/step
Epoch 4/1000  
1/1 - p0 -  [*lr: 0.0009549] - loss: 0.2289 - mae: 0.2289 - val_loss: 0.1844 - val_mae: 0.1844 - 0.04s/epoch - 0.041s/step
Epoch 5/1000  
1/1 - p0 - [*lr: 0.000000e+00] - loss: 0.1844 - mae: 0.1844 - val_loss: 0.1764 - val_mae: 0.1764 - 0.04s/epoch - 0.040s/step
Epoch 6/1000  
1/1 - p1 -  [*lr: 0.0009549] - loss: 0.1764 - mae: 0.1764 - val_loss: 0.1764 - val_mae: 0.1764 - 0.04s/epoch - 0.044s/step
Epoch 7/1000  
1/1 - p0 -  [*lr: 0.0034549] - loss: 0.1764 - mae: 0.1764 - val_loss: 0.1651 - val_mae: 0.1651 - 0.04s/epoch - 0.040s/step
Epoch 8/1000  
1/1 - p0 -  [*lr:

In [64]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 103866.65359760859, 'mape': 0.8902410664067608, 'wmape': 0.8232548950259407, 'quantile_acc': 0.725}


,date,ta,ta_lower,ta_upper
0,2021-07-16,244168.770209,101950.281427,386387.258991
1,2021-07-17,247459.417936,102975.438495,391943.397377
2,2021-07-18,146079.177971,71391.787446,220766.568496
3,2021-07-19,-128156.649283,-14042.698367,-242270.600198
4,2021-07-20,214232.556378,92624.056399,335841.056357
5,2021-07-21,142704.377553,70340.413739,215068.341367
6,2021-07-22,243940.604125,101879.199351,386002.008899
7,2021-07-23,41203.448052,38719.163487,43687.732617
8,2021-07-24,162389.384983,76473.013206,248305.756761
9,2021-07-25,171863.411313,79424.518763,264302.303863


In [65]:
test_load_and_save_model('time2vec.zip', model, scaler)

{'mae': 103866.65359760859, 'mape': 0.8902410664067608, 'wmape': 0.8232548950259407, 'quantile_acc': 0.725}


,date,ta,ta_lower,ta_upper
0,2021-07-16,244168.770209,101950.281427,386387.258991
1,2021-07-17,247459.417936,102975.438495,391943.397377
2,2021-07-18,146079.177971,71391.787446,220766.568496
3,2021-07-19,-128156.649283,-14042.698367,-242270.600198
4,2021-07-20,214232.556378,92624.056399,335841.056357
5,2021-07-21,142704.377553,70340.413739,215068.341367
6,2021-07-22,243940.604125,101879.199351,386002.008899
7,2021-07-23,41203.448052,38719.163487,43687.732617
8,2021-07-24,162389.384983,76473.013206,248305.756761
9,2021-07-25,171863.411313,79424.518763,264302.303863


## TransformerModel

In [66]:
model = TransformerModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                               quantile=quantile, enable_progress_bar=False, enable_model_summary=False)

In [67]:
model.fit(data)

In [68]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 54528.708984375, 'mape': 0.48204861970104174, 'wmape': 0.43219864158466814, 'quantile_acc': 0.275}


,date,ta,ta_lower,ta_upper
0,2021-07-16,172927.593750,135934.312500,209920.875000
1,2021-07-17,180740.375000,141611.468750,219869.281250
2,2021-07-18,189639.906250,148078.296875,231201.500000
3,2021-07-19,190496.546875,148700.781250,232292.296875
4,2021-07-20,184428.781250,144291.640625,224565.906250
5,2021-07-21,181911.656250,142462.578125,221360.734375
6,2021-07-22,190188.218750,148476.734375,231899.703125
7,2021-07-23,186205.984375,145583.031250,226828.906250
8,2021-07-24,174261.187500,136903.359375,211618.984375
9,2021-07-25,178680.515625,140114.656250,217246.343750


In [69]:
test_load_and_save_model('transformer.zip', model, scaler)

{'mae': 54528.708984375, 'mape': 0.48204861970104174, 'wmape': 0.43219864158466814, 'quantile_acc': 0.275}


,date,ta,ta_lower,ta_upper
0,2021-07-16,172927.593750,135934.312500,209920.875000
1,2021-07-17,180740.375000,141611.468750,219869.281250
2,2021-07-18,189639.906250,148078.296875,231201.500000
3,2021-07-19,190496.546875,148700.781250,232292.296875
4,2021-07-20,184428.781250,144291.640625,224565.906250
5,2021-07-21,181911.656250,142462.578125,221360.734375
6,2021-07-22,190188.218750,148476.734375,231899.703125
7,2021-07-23,186205.984375,145583.031250,226828.906250
8,2021-07-24,174261.187500,136903.359375,211618.984375
9,2021-07-25,178680.515625,140114.656250,217246.343750


## TiDE

In [70]:
model = TiDEModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                               quantile=0.9, enable_progress_bar=False, 
                 enable_model_summary=False)

In [71]:
model.fit(data)
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 25339.9080078125, 'mape': 0.21556717242632167, 'wmape': 0.2008460134641228, 'quantile_acc': 0.6}


,date,ta,ta_lower,ta_upper
0,2021-07-16,148364.859375,118694.031250,178035.671875
1,2021-07-17,123778.195312,100693.164062,146863.234375
2,2021-07-18,169536.000000,134194.281250,204877.734375
3,2021-07-19,166289.250000,131817.203125,200761.296875
4,2021-07-20,153590.140625,122519.664062,184660.593750
5,2021-07-21,151640.031250,121091.914062,182188.125000
6,2021-07-22,165002.125000,130874.843750,199129.406250
7,2021-07-23,156030.734375,124306.523438,187754.937500
8,2021-07-24,136318.328125,109874.296875,162762.359375
9,2021-07-25,133861.625000,108075.648438,159647.609375


In [72]:
test_load_and_save_model('tide.zip', model, scaler)

{'mae': 25339.9080078125, 'mape': 0.21556717242632167, 'wmape': 0.2008460134641228, 'quantile_acc': 0.6}


,date,ta,ta_lower,ta_upper
0,2021-07-16,148364.859375,118694.031250,178035.671875
1,2021-07-17,123778.195312,100693.164062,146863.234375
2,2021-07-18,169536.000000,134194.281250,204877.734375
3,2021-07-19,166289.250000,131817.203125,200761.296875
4,2021-07-20,153590.140625,122519.664062,184660.593750
5,2021-07-21,151640.031250,121091.914062,182188.125000
6,2021-07-22,165002.125000,130874.843750,199129.406250
7,2021-07-23,156030.734375,124306.523438,187754.937500
8,2021-07-24,136318.328125,109874.296875,162762.359375
9,2021-07-25,133861.625000,108075.648438,159647.609375


## SegRNN

In [73]:
model = SegRNNModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                     quantile=quantile, accelerator=accelerator, patience=100, verbose=True)
model.fit(data)

MPS  available: True   | MPS  use: False
CUDA available: False  | CUDA use: False
TPU  available: False  | TPU  use: False
CPU  available: True   | CPU  use: True


Epoch 1/1000  
1/1 - p0 -  [*lr: 0.0009045] - loss: 0.1637 - mae: 0.1637 - val_loss: 0.1419 - val_mae: 0.1419 - 0.02s/epoch - 0.023s/step
Epoch 2/1000  
1/1 - p0 -  [*lr: 0.0006545] - loss: 0.1419 - mae: 0.1419 - val_loss: 0.1228 - val_mae: 0.1228 - 0.02s/epoch - 0.022s/step
Epoch 3/1000  
1/1 - p0 -  [*lr: 0.0003455] - loss: 0.1228 - mae: 0.1228 - val_loss: 0.1095 - val_mae: 0.1095 - 0.02s/epoch - 0.023s/step
Epoch 4/1000  
1/1 - p0 -  [*lr: 9.55e-05] - loss: 0.1095 - mae: 0.1095 - val_loss: 0.1028 - val_mae: 0.1028 - 0.02s/epoch - 0.020s/step
Epoch 5/1000  
1/1 - p0 - [*lr: 0.000000e+00] - loss: 0.1028 - mae: 0.1028 - val_loss: 0.1010 - val_mae: 0.1010 - 0.02s/epoch - 0.020s/step
Epoch 6/1000  
1/1 - p1 -  [*lr: 9.55e-05] - loss: 0.1010 - mae: 0.1010 - val_loss: 0.1010 - val_mae: 0.1010 - 0.02s/epoch - 0.022s/step
Epoch 7/1000  
1/1 - p0 -  [*lr: 0.0003455] - loss: 0.1010 - mae: 0.1010 - val_loss: 0.0992 - val_mae: 0.0992 - 0.02s/epoch - 0.021s/step
Epoch 8/1000  
1/1 - p0 -  [*lr: 0

In [74]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 27744.784176144003, 'mape': 0.24482855609799703, 'wmape': 0.21990724253943522, 'quantile_acc': 0.275}


,date,ta,ta_lower,ta_upper
0,2021-07-16,130727.846666,115550.211513,145905.481819
1,2021-07-17,137140.516298,120917.823858,153363.208738
2,2021-07-18,135686.985302,119701.171299,151672.799305
3,2021-07-19,136362.511267,120266.608429,152458.414106
4,2021-07-20,137887.725112,121543.261856,154232.188367
5,2021-07-21,138604.983121,122143.630059,155066.336183
6,2021-07-22,141380.201578,124466.577865,158293.825291
7,2021-07-23,143283.587786,126059.773819,160507.401754
8,2021-07-24,144878.698203,127394.932942,162362.463464
9,2021-07-25,147635.944814,129702.837730,165569.051898


In [75]:
test_load_and_save_model('seg_rnn.zip', model, scaler)

{'mae': 27744.784176144003, 'mape': 0.24482855609799703, 'wmape': 0.21990724253943522, 'quantile_acc': 0.275}


,date,ta,ta_lower,ta_upper
0,2021-07-16,130727.846666,115550.211513,145905.481819
1,2021-07-17,137140.516298,120917.823858,153363.208738
2,2021-07-18,135686.985302,119701.171299,151672.799305
3,2021-07-19,136362.511267,120266.608429,152458.414106
4,2021-07-20,137887.725112,121543.261856,154232.188367
5,2021-07-21,138604.983121,122143.630059,155066.336183
6,2021-07-22,141380.201578,124466.577865,158293.825291
7,2021-07-23,143283.587786,126059.773819,160507.401754
8,2021-07-24,144878.698203,127394.932942,162362.463464
9,2021-07-25,147635.944814,129702.837730,165569.051898


# MultiOutputRegressorModel

In [76]:
from PipelineTS.ml_model import MultiOutputRegressorModel

In [77]:
from xgboost import XGBRegressor

model = MultiOutputRegressorModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                     quantile=quantile, estimator=XGBRegressor)
model.fit(data, fit_kwargs={'verbose': False})

In [78]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 35059.616942536086, 'mape': 0.30956226420304744, 'wmape': 0.2778851562648378, 'quantile_acc': 0.4}


,date,ta,ta_lower,ta_upper
0,2021-07-16,140113.404607,112409.717398,167817.091816
1,2021-07-17,174127.563922,137232.184412,211022.943432
2,2021-07-18,187086.737532,146689.383000,227484.092063
3,2021-07-19,156392.433602,124289.642359,188495.224846
4,2021-07-20,146195.197282,116848.019009,175542.375556
5,2021-07-21,137465.693534,110477.500871,164453.886197
6,2021-07-22,137051.004778,110174.874019,163927.135537
7,2021-07-23,142549.612415,114187.585473,170911.639356
8,2021-07-24,148693.808848,118671.427390,178716.190307
9,2021-07-25,158354.841917,125721.746419,190987.937415


In [79]:
test_load_and_save_model('mor.zip', model, scaler)

{'mae': 35059.616942536086, 'mape': 0.30956226420304744, 'wmape': 0.2778851562648378, 'quantile_acc': 0.4}


,date,ta,ta_lower,ta_upper
0,2021-07-16,140113.404607,112409.717398,167817.091816
1,2021-07-17,174127.563922,137232.184412,211022.943432
2,2021-07-18,187086.737532,146689.383000,227484.092063
3,2021-07-19,156392.433602,124289.642359,188495.224846
4,2021-07-20,146195.197282,116848.019009,175542.375556
5,2021-07-21,137465.693534,110477.500871,164453.886197
6,2021-07-22,137051.004778,110174.874019,163927.135537
7,2021-07-23,142549.612415,114187.585473,170911.639356
8,2021-07-24,148693.808848,118671.427390,178716.190307
9,2021-07-25,158354.841917,125721.746419,190987.937415


# MultiStepRegressorModel

In [80]:
from PipelineTS.ml_model.multi_output_model import MultiStepRegressorModel
from xgboost import XGBRegressor

model = MultiStepRegressorModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                     quantile=quantile, estimator=XGBRegressor)
model.fit(data, fit_kwargs={'verbose': False})

In [81]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 47735.35705996379, 'mape': 0.41989739109615043, 'wmape': 0.37835402416710856, 'quantile_acc': 0.425}


,date,ta,ta_lower,ta_upper
0,2021-07-16,140113.404607,108316.529065,171910.280148
1,2021-07-17,126370.383071,98836.011877,153904.754265
2,2021-07-18,100734.028841,81150.971052,120317.086631
3,2021-07-19,110859.391413,88135.874165,133582.908662
4,2021-07-20,120839.138398,95020.325490,146657.951307
5,2021-07-21,115277.022512,91183.342835,139370.702190
6,2021-07-22,116462.516652,92001.146805,140923.886499
7,2021-07-23,138577.929902,107257.293708,169898.566097
8,2021-07-24,138277.385549,107049.965509,169504.805590
9,2021-07-25,139179.352683,107672.180565,170686.524801


In [82]:
test_load_and_save_model('msr.zip', model, scaler)

{'mae': 47735.35705996379, 'mape': 0.41989739109615043, 'wmape': 0.37835402416710856, 'quantile_acc': 0.425}


,date,ta,ta_lower,ta_upper
0,2021-07-16,140113.404607,108316.529065,171910.280148
1,2021-07-17,126370.383071,98836.011877,153904.754265
2,2021-07-18,100734.028841,81150.971052,120317.086631
3,2021-07-19,110859.391413,88135.874165,133582.908662
4,2021-07-20,120839.138398,95020.325490,146657.951307
5,2021-07-21,115277.022512,91183.342835,139370.702190
6,2021-07-22,116462.516652,92001.146805,140923.886499
7,2021-07-23,138577.929902,107257.293708,169898.566097
8,2021-07-24,138277.385549,107049.965509,169504.805590
9,2021-07-25,139179.352683,107672.180565,170686.524801
